<a href="https://colab.research.google.com/github/Josh-Tang112/Map_Marker_w_Photo/blob/main/Marker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

In [7]:
import os
import statistics

In [8]:
flst = os.listdir('.')
flst = [f for f in flst if f.lower().endswith('.jpg')]
flst

['IMG_2591.JPG', 'IMG_2872.JPG']

In [ ]:
!ls -l

total 3828
-rw-r--r-- 1 root root 2246370 Jun  3 16:02 Rename.JPG
-rw-r--r-- 1 root root 1670371 Jun  3 16:02 Test.JPG


In [ ]:
dmt = { TAGS[k]: v for k, v in Image.open(flst[0])._getexif().items() if k in TAGS }["GPSInfo"]
dmt

{1: 'N',
 2: (34.0, 51.0, 49.6),
 3: 'W',
 4: (82.0, 22.0, 52.11),
 5: b'\x00',
 6: 283.6512195121951,
 7: (14.0, 9.0, 0.0),
 12: 'K',
 13: 0.0,
 16: 'T',
 17: 25.735258724428398,
 23: 'T',
 24: 25.735258724428398,
 29: '2017:08:20',
 31: 5.0}

In [ ]:
gpsinfo = {}
for key in dmt.keys():
    decode = GPSTAGS.get(key,key)
    gpsinfo[decode] = dmt[key]
gpsinfo

{'GPSLatitudeRef': 'N',
 'GPSLatitude': (34.0, 51.0, 49.6),
 'GPSLongitudeRef': 'W',
 'GPSLongitude': (82.0, 22.0, 52.11),
 'GPSAltitudeRef': b'\x00',
 'GPSAltitude': 283.6512195121951,
 'GPSTimeStamp': (14.0, 9.0, 0.0),
 'GPSSpeedRef': 'K',
 'GPSSpeed': 0.0,
 'GPSImgDirectionRef': 'T',
 'GPSImgDirection': 25.735258724428398,
 'GPSDestBearingRef': 'T',
 'GPSDestBearing': 25.735258724428398,
 'GPSDateStamp': '2017:08:20',
 'GPSHPositioningError': 5.0}

In [9]:
def get_GPS(iname):
    image = Image.open(iname)
    exif = { TAGS[k]: v for k, v in image._getexif().items() if k in TAGS } # get metadata

    # flip image if needed and update orientation, check http://sylvana.net/jpegcrop/exif_orientation.html for details
    need_flip = [2,4,5,7]
    correspond_orientation = [1,3,8,6]
    mapping = res = dict(zip(need_flip, correspond_orientation))
    if exif['Orientation'] in need_flip:
        image = image.transpose(Image.FLIP_LEFT_RIGHT)
        exif['Orientation'] = mapping(exif['Orientation'])

    # rotate image if needed
    if exif['Orientation'] == 3:
        image = image.rotate(180, expand=True)
    elif exif['Orientation'] == 6:
        image = image.rotate(270, expand=True)
    elif exif['Orientation'] == 8:
        image = image.rotate(90, expand=True)
        
    image = image.resize((350, 350), Image.ANTIALIAS)
    image.save(iname, 'jpeg', quality=100)
    return exif['GPSInfo']

In [10]:
def convert_to_degress(value):
    return value[0] + (value[1] / 60.0) + (value[2] / 3600.0)

def get_coord(gps):
    latitude = gps[2]
    latitude_ref = gps[1]
    longitude = gps[4]
    longitude_ref = gps[3]
    if latitude:
        lat_value = convert_to_degress(latitude)
        if latitude_ref != 'N':
            lat_value = -lat_value
    else:
        return {}
    if longitude:
        lon_value = convert_to_degress(longitude)
        if longitude_ref != 'E':
            lon_value = -lon_value
    else:
        return {}
    return (lat_value, lon_value)

In [12]:
lst_coord = [get_coord(get_GPS(f)) for f in flst]
lst_coord

[(34.86377777777778, -82.38114166666666), (34.871180555555554, -82.372575)]

In [13]:
center = (statistics.mean([i[0] for i in lst_coord]), statistics.mean([i[1] for i in lst_coord]))
center

(34.86747916666667, -82.37685833333333)

In [14]:
import folium
import base64
from folium import IFrame

In [15]:
m = folium.Map(location = center, zoom_start = 6, tiles = "OpenStreetMap")
# m

In [16]:
def add_to_map(m,iname,coord):
    encoded = base64.b64encode(open(iname, 'rb').read())
    html = '<img src="data:image/png;base64,{}">'.format
    iframe = IFrame(html(encoded.decode('UTF-8')), width=370, height=370)

    popup = folium.Popup(iframe, max_width=550)
    icon=folium.Icon(color = 'gray',icon='globe')
    folium.Marker(location=coord, popup = popup, icon=icon).add_to(m)

In [17]:
for i in range(len(flst)):
    add_to_map(m,flst[i],lst_coord[i])
m